In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from googlesearch import search

from bs4 import BeautifulSoup
import pandas as pd
from math import nan
from datetime import datetime
from requests import get



In [2]:
# Define a test user-agent string to simulate a browser request
test_ua = 'Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36'

# Initialize Chrome options for the WebDriver
options = Options()

# Uncomment the following line if you want to run the browser in headless mode (no GUI)
# options.add_argument("--headless")

# Set the window size for the browser
options.add_argument("--window-size=1920,1080")

# Set the user-agent to the test user-agent defined above
options.add_argument(f'--user-agent={test_ua}')

# Add additional options to improve stability and compatibility
options.add_argument('--no-sandbox')  # Required for running in some environments
options.add_argument("--disable-extensions")  # Disable extensions to avoid potential conflicts
options.add_argument("user-data-dir=C:\\Users\\eric9\\AppData\\Local\\Google\\Chrome\\User Data\\Default")

# Initialize the Chrome WebDriver with the specified options
driver = webdriver.Chrome(options=options)

In [3]:
driver.get("https://stubhub.com/")

In [71]:
from configparser import SectionProxy
from bs4 import BeautifulSoup
from regex import D



ps = driver.page_source
soup = BeautifulSoup(ps, 'html.parser')

# print (soup.prettify())
# Parse the HTML element with BeautifulSoup
sales = soup.find_all('tr', class_='sc-1yc2nz5-10 ctyECr')

# print(sales[0].prettify())
# Create an empty list to store the extracted data
data = []

for sale in sales:
    # Extract the required information
    try:
        event_name = sale.select_one('a.sc-hjQCSK.gcdlOK').text.strip()
        event_date = datetime.strptime(sale.select('div.sc-hKMtZM.sc-xr8ck3-6.biNfYF.loESia')[0].text.strip(), '%a %b %d %Y (event local time)')
        # seats_delivery = sale.select('td.sc-1yc2nz5-11.boFgXL')[1].text.strip()'"Ozone Research"
        venue = sale.select('a.sc-hjQCSK.gcdlOK')[1].text.strip().split(',')[0]
        location = sale.select('a.sc-hjQCSK.gcdlOK')[1].text.strip().split(',')[1]
        quantity = int(sale.select('td.sc-1yc2nz5-11.boFgXL')[1].select('div.sc-xr8ck3-0.kWnfZs > div')[0].text.strip())
        section = sale.select('td.sc-1yc2nz5-11.boFgXL')[1].select('div.sc-xr8ck3-0.kWnfZs > div')[1].text.strip() 
        section_info = section.split('|')
        section = section_info[0].strip()
        if len(section_info) > 2:
            row = section_info[1].strip()
            seat = section_info[2].strip()
        else:
            row = nan
            seat = nan
        sale_date = datetime.strptime(sale.select('td.sc-1yc2nz5-11.boFgXL')[2].text.strip(), '%A, %d %B %Y(event local time)')
        total_payout = float(sale.select('td.sc-1yc2nz5-11.boFgXL')[4].text.strip().replace('C$', '').replace(',', ''))

        # Append the extracted information to the data list
        data.append({
            'Event Name': event_name,
            'Event Date': event_date,
            'Venue': venue,
            'Quantity': quantity,
            'Section': section,
            'Row': row,
            'Seat': seat,
            'Sale Date': sale_date,
            'Total Payout': total_payout
        })
    except Exception as e:
        print(f"Error processing sale: {e}")
        pass

# Convert the data list to a DataFrame
df_sales = pd.DataFrame(data)

# Convert datetime columns to date
df_sales['Event Date'] = df_sales['Event Date'].dt.date
df_sales['Sale Date'] = df_sales['Sale Date'].dt.date
# Display the DataFrame
print(df_sales)


Error processing sale: 'NoneType' object has no attribute 'text'
           Event Name  Event Date                        Venue  Quantity  \
0       Timmy Trumpet  2024-11-01  Rebel Entertainment Complex         2   
1       Timmy Trumpet  2024-11-01  Rebel Entertainment Complex         2   
2       Timmy Trumpet  2024-11-01  Rebel Entertainment Complex         2   
3       Timmy Trumpet  2024-11-01  Rebel Entertainment Complex         2   
4   Maddix with FOVOS  2024-11-08               Forbidden City         2   
5   Maddix with FOVOS  2024-11-08               Forbidden City         1   
6       Sullivan King  2024-11-15  Rebel Entertainment Complex         1   
7       Sullivan King  2024-11-15  Rebel Entertainment Complex         3   
8       Sullivan King  2024-11-15  Rebel Entertainment Complex         2   
9        Nora En Pure  2024-12-27                      History         2   
10        Alan Walker  2025-02-07  Rebel Entertainment Complex         1   
11        Alan Walker  

In [72]:
# # Save the DataFrame to the Excel file, replacing the existing sheet if it exists
path = "../Documents/Ticket Sales.xlsx"

with pd.ExcelWriter(path, mode='a', engine="openpyxl", if_sheet_exists="replace") as writer:
    df_sales.to_excel(writer, sheet_name="Sales 2.0", index=False)

In [75]:
# Group by 'Event Name', 'Event Date', and 'Section', then sum 'Quantity' and 'Total Payout'
grouped_df = df_sales.groupby(['Event Name', 'Event Date', 'Venue']).agg({
    'Quantity': 'sum',
    'Total Payout': 'sum'
}).reset_index()

# Display the grouped DataFrame
print(grouped_df)


                                           Event Name  Event Date  \
0                                             50 Cent  2024-07-20   
1                                             50 Cent  2024-07-20   
2                                         Alan Walker  2025-02-07   
3                                            Coldplay  2025-07-12   
4                                            Crankdat  2024-09-06   
5                                         DANNY AVILA  2024-10-04   
6                  DJ Diesel, A.K.A. Shaquille O'Neal  2024-09-28   
7              Electric Island: Sara Landry and Yotto  2024-08-10   
8                                            Excision  2024-03-15   
9                                         GORGON CITY  2024-07-07   
10                                           Galantis  2024-05-25   
11                                           Illenium  2024-09-01   
12                                           Illenium  2024-09-02   
13                                

In [59]:
path = "../Documents/Ticket Sales.xlsx"
events = pd.read_excel(path, sheet_name ="My Profits")


In [60]:
mysale = events  
mysale['Event Date'] = pd.to_datetime(mysale['Date']).dt.date
mysale['Event Name'] = events['Artist']
mysale['Venue'] = events['Venue']
mysale['Quantity'] = events['Sold']
mysale['Total Payout'] = events['Sold'] * events['Resell']


In [69]:
# df_sales1 = df_sales
# grouped_df1 = grouped_df


In [74]:
# df_sales1 = df_sales
# grouped_df1 = grouped_df
df_sales = pd.concat([df_sales, df_sales1], ignore_index=True)



In [81]:

from hmac import new


new_sale = []


for index1, row1 in mysale.iterrows():
    no_stub = True
    for index2, row2 in grouped_df.iterrows():
        if row1['Event Name'].lower() in row2['Event Name'].lower() and row1['Event Date'] == row2['Event Date'] and (row1['Venue'].lower()[0] in row2['Venue'].lower()[0] or row1['Venue'].lower() in row2['Venue'].lower()) :
            no_stub = False
            if row1['Quantity'] > row2['Quantity']:
                
                grouped_df.at[index2, 'Evemt Name'] = row1['Event Name']
                grouped_df.at[index2, 'Venue'] = row1['Venue']
                new_sale.append({
                    'Event Name': row1['Event Name'],
                    'Event Date': row1['Event Date'],
                    'Venue': row1['Venue'],
                    'Quantity': row1['Quantity'] - row2['Quantity'],
                    'Total Payout': row1['Total Payout'] - row2['Total Payout']
                })
            break
    if no_stub:
        if row1['Quantity'] > 0:
            new_sale.append({
                'Event Name': row1['Event Name'],
                'Event Date': row1['Event Date'],
                'Venue': row1['Venue'],
                'Quantity': row1['Quantity'],
                'Total Payout': row1['Total Payout']
            })
    
            
        

new_sale_df = pd.DataFrame(new_sale)

In [82]:
new_sale_df

,Event Name,Event Date,Venue,Quantity,Total Payout
0,Day 1 DPRTMNT AP,2024-08-02,NaN,4,0.000000
1,Day 1 Rebel AP,2024-08-02,NaN,4,200.000000
2,Dom Dolla,2024-09-06,Budweiser,6,720.000000
3,Excision,2024-03-15,Casino,1,112.900000
4,Illenium,2024-09-01,Rebel,11,1264.820000
5,Illenium,2024-09-02,Rebel,1,70.000000
6,Inquisitive,2024-07-12,Annabels,6,390.000000
7,John Sumit,2024-06-15,Budweiser,6,681.000000
8,Party NextDoor,2024-08-03,Cabana,4,152.000000
9,seven lions,2024-08-30,Rebel,2,131.620000


In [83]:
final = pd.concat([df_sales, new_sale_df], ignore_index=True)
final

,Event Name,Event Date,Venue,Quantity,Section,Row,Seat,Sale Date,Total Payout
0,Timmy Trumpet,2024-11-01,Rebel Entertainment Complex,2,General Admission,NaN,NaN,2024-10-07,146.720000
1,Timmy Trumpet,2024-11-01,Rebel Entertainment Complex,2,General Admission,NaN,NaN,2024-10-07,146.720000
2,Timmy Trumpet,2024-11-01,Rebel Entertainment Complex,2,General Admission,NaN,NaN,2024-09-20,146.720000
3,Timmy Trumpet,2024-11-01,Rebel Entertainment Complex,2,General Admission,NaN,NaN,2024-09-06,179.980000
4,Maddix with FOVOS,2024-11-08,Forbidden City,2,General Admission,NaN,NaN,2024-10-07,161.120000
...,...,...,...,...,...,...,...,...,...
112,Testpilot x Jauz,2024-06-01,Rebel,1,NaN,NaN,NaN,NaN,55.000000
113,Timmy Trumpet,2024-11-01,Rebel,4,NaN,NaN,NaN,NaN,220.034545
114,Tyga,2024-08-17,Cabana,1,NaN,NaN,NaN,NaN,-0.700000
115,Veld Day 1,2024-08-02,NaN,5,NaN,NaN,NaN,NaN,1200.000000


In [84]:
with pd.ExcelWriter(path, mode='a', engine="openpyxl", if_sheet_exists="replace") as writer:
    final.to_excel(writer, sheet_name="Sales 2.0", index=False)

: 